In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from webdriver_manager.firefox import GeckoDriverManager
from bs4 import BeautifulSoup as bsp
import pandas as pd
import time, random

In [2]:
URL = "https://eumostwanted.eu/es"

In [3]:
def waitFinishLoad(driver, byLoader=By.ID, valueLoader=None, dissapear=False, extra_wait=False):
    if valueLoader is None: return
    WebDriverWait(driver, 10).until(EC.presence_of_element_located((byLoader, valueLoader)))
    if dissapear: WebDriverWait(driver, 10).until(EC.invisibility_of_element((byLoader, valueLoader)))
    if extra_wait: time.sleep(random.choice([0.6, 0.5, 0.8, 0.7, 0.9, 0.4]))

def get_elem(driver, by, value):
    return driver.find_element(by, value)

def get_elements(driver, by, value):
    return driver.find_elements(by, value)

def click(driver, by, value):
    get_elem(driver, by, value).click()

def click_script(driver, value, by=By.ID):
    driver.execute_script(get_elem(driver, by, value).get_dom_attribute("onClick"))

In [4]:
GeckoDriverManager().install()
driver = webdriver.Firefox()

In [5]:
test = []
try:
    driver.get(URL)
    waitFinishLoad(driver, byLoader=By.XPATH, valueLoader="//script[contains(@type, 'application/json')]")
    click(driver, By.ID, "sort_checkbox")
    
    for i in range(len(get_elements(driver, By.XPATH, "//div[contains(@class, 'wantedItem')]"))):

        if "arrested" not in get_elem(driver, By.XPATH,f"//div[contains(@class, 'wantedItem')][{i+1}]").get_attribute("class"):
            click(driver, By.XPATH,f"//div[contains(@class, 'wantedItem')][{i+1}]")
            test.append(get_elem(driver, By.CLASS_NAME,"wanted_top_right").text)
            click(driver, By.CLASS_NAME, "mobile")
        
except NoSuchElementException as e:
    print("Error\n", e)

In [6]:
criminals = []
for i in range(len(test)):
    nombre = test[i].split("\n")[0].split("Wanted")[0].split("PELIGROSO")[0] 
    for linea in test[i].split("\n"):
        if "by" in linea:
            if "by" in linea.split("Wanted")[0]:
                buscado = linea.split("Wanted")[0].split("by")[1]
            else:
                buscado = linea.split("Wanted")[1].split("by")[1]
        if "FECHA" in linea:
            fecha = linea.split("NACIMIENTO")[1]
            edad = linea.split("NACIMIENTO")[1].split("(")[1].split(" ")[0]
        if "NACIONALIDAD" in linea:
            nacionalidad = linea.split("NACIONALIDAD")[1]
        if "ORIGEN" in linea:
            origen = linea.split("ÉTNICO")[1]
        if "IDIOMAS" in linea:
            idiomas = linea.split("HABLADOS")[1]
    
    criminals.append({"nombre": nombre, "buscado": buscado, "fecha_nacimiento": fecha, "edad": edad, "nacionalidad": nacionalidad, "origen_etnico": origen, "idiomas": idiomas})

In [7]:
df = pd.DataFrame(criminals)
df.head()

,nombre,buscado,fecha_nacimiento,edad,nacionalidad,origen_etnico,idiomas
0,"GARWEG, Burkhard",Alemania,"Septiembre 1, 1968 (55 years)",55,German,European,German
1,"STAUB, Ernst-Volker",Alemania,"Octubre 30, 1954 (69 years)",69,German,European,German
2,"JANKEVICS, Māris",Letonia,"Enero 24, 1979 (45 years)",45,Latvian,European,German
3,"BURRELI, Aleks Καταζητούμενος by Κύπρος",Κύπρος,"Febrero 1, 1968 (56 years)",56,Latvian,Ευρώπης,Albanian
4,"LEIJDEKKERS, Joseph Johannes (Jos)",Holanda,"Julio 1, 1991 (32 years)",32,Dutch,European,Albanian


In [8]:
df["edad"] = df["edad"].astype("Int64")

# Cantos fuxitivos busca cada país.

In [9]:
df.groupby(["buscado"])["buscado"].count()

buscado
 Alemania           2
 Austria            2
 Bulgaria           2
 Bélgica            2
 Chipre             1
 Croacia            1
 Dinamarca          2
 Eslovaquia         3
 Eslovenia          2
 España             2
 Estonia            3
 Finlandia          2
 Francia            3
 Holanda            3
 Hungría            2
 Italia             2
 Letonia            2
 Lituania           3
 Luxemburgo         2
 Malta              2
 Polonia            2
 República Checa    3
 Rumanía            2
 Suecia             3
 Κύπρος             1
an                  2
Name: buscado, dtype: int64

# A idade media, a mediana da idade e os valores extremos

In [10]:
df["edad"].agg(["mean","median","min","max"])

mean      45.053571
median    45.000000
min       23.000000
max       74.000000
Name: edad, dtype: float64